In [184]:
import json
import string
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory


import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to C:\Users\Judan Syamsul
[nltk_data]     Hadad\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [185]:
# Load json file
f = open('intent/intent.json', 'r')
intent_json = json.load(f)

In [186]:
# Create list from json
input = []
intent = []

for i in range(len(intent_json['intents'])):
    for user_input in intent_json['intents'][i]['input']:
        input.append(user_input)
        intent.append(intent_json['intents'][i]['intent'])

In [187]:
# Create dataframe from json
df = pd.DataFrame({
    'input': input,
    'intent' : intent
    # 'response' : response
})
df.head()

,input,intent
0,halo,sapa
1,hei,sapa
2,hey,sapa
3,hello,sapa
4,hallo,sapa


In [188]:
# Define stemmer and stopword remover
factory = StemmerFactory()
stemmer = factory.create_stemmer()

factory = StopWordRemoverFactory()
stopwords = factory.get_stop_words()

In [189]:
slang = pd.read_csv('lexicon/slang ke semi baku.csv')

slang_replace = {}
for i, row in enumerate(slang['slang']):
    slang_replace[row] = slang['formal'].iloc[i]

In [190]:
baku = pd.read_csv('lexicon/slang ke baku.csv')

baku_replace = {}
for i, row in enumerate(baku['slang']):
    baku_replace[row] = baku['baku'].iloc[i]

In [191]:
stopwords.remove('ok')
stopwords.remove('tidak')

In [193]:
# Create text cleaning function
def clean_text(text):
    new_text = []

    text = text.lower()

    for kata in text.split():
        if kata not in (slang_replace|baku_replace):
            new_text.append(kata)
        elif kata in baku_replace:
            new_text+=baku_replace[kata].split()
        elif kata in slang_replace:
            new_text+=slang_replace[kata].split()
    new_text = ' '.join(
        stemmer.stem(
            baku_replace.get(
                word,
                word
            )
        ) for word in new_text if word not in stopwords
    )

    new_text = new_text.translate(
        str.maketrans(
            '',
            '',
            string.punctuation
        )
    )
    
    return new_text

In [194]:
kalimat = 'Aku laper banget gatau maunya makan jeruk apa lagi meniru-niru... daah ngelamar'
clean_text(kalimat)

'aku lapar banget tidak tahu mau makan jeruk apa tiru deh lamar'

In [195]:
df['clean_input'] = df['input'].apply(clean_text)
df['clean_input'].head()

0     halo
1      hei
2      hai
3    hello
4     halo
Name: clean_input, dtype: object

In [196]:
# Create corpus
words = set([
    word for word in df['clean_input'] for word in word_tokenize(word)
    ]
)

In [197]:
corpus_size = len(words)

In [198]:
# Calculate word length of each row
df['length'] = df['clean_input'].apply(word_tokenize).apply(len)

In [199]:
sequence_length = int(round(df['length'].max(),0))

In [200]:
corpus_size, sequence_length

(302, 8)

In [201]:
# Label encoding
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

le = LabelEncoder()
y_train = le.fit_transform(df['intent'])
y_train = to_categorical(y_train)

In [202]:
le.classes_

array(['CV', 'biaya', 'daftar', 'dokumen', 'error upload', 'ktp',
       'link sosmed', 'lowongan', 'lupa password', 'nama', 'pas foto',
       'pengantar', 'penutup', 'qualification', 'responsibilities',
       'salary', 'sapa', 'sertifikat', 'skck', 'timeline', 'training',
       'transkrip-ijazah'], dtype=object)

In [203]:
len(le.classes_)

22

In [204]:
textvect = tf.keras.layers.TextVectorization(
    max_tokens=corpus_size,
    standardize='lower_and_strip_punctuation',
    split='whitespace',
    ngrams=None,
    output_mode='int',
    output_sequence_length=sequence_length
)
textvect.adapt(df['clean_input'])

In [205]:
embedding = tf.keras.layers.Embedding(
    input_dim=corpus_size,
    output_dim=16,
    input_length=sequence_length,
    embeddings_initializer='uniform'
)

In [206]:
tes = 'saya mau daftar rekrutmen'
textvect(clean_text(tes))

<tf.Tensor: shape=(8,), dtype=int64, numpy=array([19, 13,  9,  0,  0,  0,  0,  0], dtype=int64)>

In [207]:
embedding(textvect(clean_text(tes)))

<tf.Tensor: shape=(8, 16), dtype=float32, numpy=
array([[ 0.02602616, -0.00233669, -0.00157882,  0.0107664 ,  0.01552376,
         0.02170381,  0.01976177, -0.01147447,  0.00224482,  0.04782111,
        -0.04961878, -0.02931265,  0.00936665, -0.00975845,  0.0277995 ,
         0.01213394],
       [ 0.01847312, -0.04169055,  0.00519992, -0.02603018, -0.02136955,
        -0.02666293,  0.02691633,  0.02731865, -0.0304291 , -0.02631148,
         0.04042557,  0.03144511,  0.00719743, -0.0259292 ,  0.01389143,
         0.00269145],
       [-0.03238012, -0.01309792,  0.01792368,  0.04275905, -0.02144547,
         0.03008294, -0.01082361,  0.03866122, -0.01051319,  0.01639554,
        -0.03229091, -0.00112914, -0.00718968,  0.03750296,  0.00700215,
        -0.03392418],
       [ 0.03017514,  0.01008932,  0.02843987,  0.04292668,  0.03386657,
         0.04759934, -0.03848273,  0.03099462, -0.00232458, -0.03826989,
        -0.0118816 , -0.010799  , -0.01647909, -0.01562829,  0.03071154,
         

In [208]:
# Create model
input = tf.keras.layers.Input(shape=(1,), dtype='string')
hidden_layer = textvect(input)
hidden_layer = embedding(hidden_layer)
hidden_layer = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16))(hidden_layer)
output = tf.keras.layers.Dense(len(le.classes_), activation='softmax')(hidden_layer)
model = tf.keras.Model(inputs=input, outputs=output)

# Compile model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['categorical_accuracy']
)

In [209]:
hist = model.fit(df['clean_input'], y_train, epochs=100, verbose=0)
pd.DataFrame(hist.history).tail()

,loss,categorical_accuracy
95,0.023880,1.0
96,0.023373,1.0
97,0.023037,1.0
98,0.022409,1.0
99,0.021866,1.0


In [210]:
model.evaluate(df['clean_input'], y_train)

15/15 [==============================] - 1s 1ms/step - loss: 0.0213 - categorical_accuracy: 1.0000


[0.02132023498415947, 1.0]

In [211]:
y_train_df = pd.DataFrame(y_train, columns=le.classes_)
y_train_df['intent'] = y_train_df.idxmax(axis=1)

model_pred = model.predict(df['clean_input'])
model_pred = pd.DataFrame(model_pred, columns=le.classes_)
model_pred['intent'] = model_pred.idxmax(axis=1)

from sklearn.metrics import classification_report
print(classification_report(y_train_df['intent'], model_pred['intent']))

                  precision    recall  f1-score   support

              CV       1.00      1.00      1.00        12
           biaya       1.00      1.00      1.00        27
          daftar       1.00      1.00      1.00        22
         dokumen       1.00      1.00      1.00        23
    error upload       1.00      1.00      1.00         9
             ktp       1.00      1.00      1.00        16
     link sosmed       1.00      1.00      1.00        17
        lowongan       1.00      1.00      1.00        17
   lupa password       1.00      1.00      1.00        21
            nama       1.00      1.00      1.00         8
        pas foto       1.00      1.00      1.00        16
       pengantar       1.00      1.00      1.00        10
         penutup       1.00      1.00      1.00        28
   qualification       1.00      1.00      1.00        18
responsibilities       1.00      1.00      1.00        21
          salary       1.00      1.00      1.00        52
            s

In [212]:
def bot_response(text):
    """Take text as function input then predict using model. Return response based on highest probability using numpy argmax    
    """
    text = clean_text(text)
    pred = model.predict([text])
    res = le.classes_[pred.argmax()]
    try:
        if textvect(text).numpy().max() > 1:
            for label_pred in intent_json['intents']:
                if label_pred['intent'] == res:
                    response = label_pred['response']
        else:
            response = ['Maaf, saya tidak mengerti']
    except:
        response = ['Maaf, saya tidak mengerti']

    dict_temp = []
    for i in range(len(pred[0])):
        temp = {le.classes_[i]: pred[0][i]}
        dict_temp.append(temp)
    print(dict_temp)
    print(le.classes_[pred.argmax()])
    return print(np.random.choice(response))

In [213]:
import pickle
pickle.dump(le, open('saved_model/encoder.pkl', 'wb'))

In [214]:
pickle.dump({'config': textvect.get_config(),
             'weights': textvect.get_weights()}
            , open("saved_model/textvect.pkl", "wb"))

In [215]:
model.save('saved_model/model', save_traces=True)

INFO:tensorflow:Assets written to: saved_model/model\assets


INFO:tensorflow:Assets written to: saved_model/model\assets


In [216]:
textvect('ok')

<tf.Tensor: shape=(8,), dtype=int64, numpy=array([225,   0,   0,   0,   0,   0,   0,   0], dtype=int64)>

In [217]:
clean_text('ok')

'ok'

In [218]:
textvect('ok').numpy().max() > 1

True

In [219]:
bot_response('ok')

[{'CV': 6.064847e-05}, {'biaya': 0.0025005913}, {'daftar': 1.6737331e-05}, {'dokumen': 0.007230959}, {'error upload': 9.070777e-07}, {'ktp': 1.0782406e-05}, {'link sosmed': 0.0007616811}, {'lowongan': 0.0020088304}, {'lupa password': 0.0001566622}, {'nama': 3.1499226e-07}, {'pas foto': 1.6106223e-05}, {'pengantar': 6.570527e-07}, {'penutup': 0.98323405}, {'qualification': 0.0006069163}, {'responsibilities': 0.00021616495}, {'salary': 0.0004097949}, {'sapa': 0.0012286727}, {'sertifikat': 2.2347885e-07}, {'skck': 1.5869287e-05}, {'timeline': 0.0005565079}, {'training': 0.0001900422}, {'transkrip-ijazah': 0.0007768776}]
penutup
Terima kasih telah bertanya. Senang bisa membantumu :D


In [220]:
testext = 'jeruk'
clean_text(testext), textvect(clean_text(testext))

('jeruk',
 <tf.Tensor: shape=(8,), dtype=int64, numpy=array([1, 0, 0, 0, 0, 0, 0, 0], dtype=int64)>)

In [221]:
textvect(clean_text('laper pengen makan jeruk')).numpy().max() > 1

False

In [227]:
bot_response('ada lowongan apa nih di pt fiktif')

[{'CV': 0.0001481711}, {'biaya': 0.0018818047}, {'daftar': 1.3752714e-05}, {'dokumen': 0.026572369}, {'error upload': 1.6646602e-06}, {'ktp': 2.6151522e-06}, {'link sosmed': 0.0020583728}, {'lowongan': 0.90186024}, {'lupa password': 2.6220496e-06}, {'nama': 0.00031909734}, {'pas foto': 1.2690709e-05}, {'pengantar': 0.0017332683}, {'penutup': 0.012268006}, {'qualification': 0.0078046443}, {'responsibilities': 0.0029560975}, {'salary': 2.1275082e-05}, {'sapa': 0.0019646776}, {'sertifikat': 1.321577e-05}, {'skck': 0.00013408503}, {'timeline': 0.029027779}, {'training': 0.004458515}, {'transkrip-ijazah': 0.006744997}]
lowongan
Saat ini PT Fiktif sedang membuka lowongan untuk posisi Data Analyst, Data Scientist, dan Data Engineer. Yuk daftar lewat https://rekrutmen.fiktif.id/


In [223]:
bot_response('ingin')

[{'CV': 4.835308e-05}, {'biaya': 0.011099085}, {'daftar': 0.011152483}, {'dokumen': 0.0008918493}, {'error upload': 0.0016470931}, {'ktp': 3.9018214e-05}, {'link sosmed': 0.00041335748}, {'lowongan': 3.9918676e-05}, {'lupa password': 0.00094954704}, {'nama': 2.114538e-05}, {'pas foto': 9.8727425e-05}, {'pengantar': 2.1157604e-07}, {'penutup': 0.013870695}, {'qualification': 3.009443e-05}, {'responsibilities': 0.00048135803}, {'salary': 0.03283696}, {'sapa': 0.91152996}, {'sertifikat': 1.3491325e-06}, {'skck': 0.0009422245}, {'timeline': 0.0006764822}, {'training': 0.013119047}, {'transkrip-ijazah': 0.000111060355}]
sapa
Maaf, saya tidak mengerti


In [224]:
bot_response('pas foto hilang gimana ya')

[{'CV': 0.0007665959}, {'biaya': 0.0010134465}, {'daftar': 0.0075141564}, {'dokumen': 0.000111901616}, {'error upload': 0.00036802498}, {'ktp': 0.064586945}, {'link sosmed': 0.00013491992}, {'lowongan': 2.5624186e-06}, {'lupa password': 0.054476768}, {'nama': 0.00032472145}, {'pas foto': 0.8657897}, {'pengantar': 1.4026862e-05}, {'penutup': 0.0001404407}, {'qualification': 1.8882174e-05}, {'responsibilities': 4.4168744e-05}, {'salary': 0.0004737596}, {'sapa': 5.1739054e-05}, {'sertifikat': 0.0041019386}, {'skck': 6.151869e-05}, {'timeline': 9.2006064e-07}, {'training': 1.6406437e-06}, {'transkrip-ijazah': 1.1907014e-06}]
pas foto
Pas foto merupakan dokumen wajib pendaftaran ya, Kak.

Berikut ketentuan pas foto yang harus dipenuhi:
- Background berwarna biru dengan pakaian formal (kemeja dan/atau jas)
- Foto berukuran 3x4 dan berwarna
- Disarankan untuk menggunakan foto terbaru
- Upload foto di https://rekrutmen.fiktif.id/dokumen dengan ukuran file tidak lebih dari 1 MB dengan format fi

ktp, pas foto, password masih ketuker2

In [225]:
le.classes_

array(['CV', 'biaya', 'daftar', 'dokumen', 'error upload', 'ktp',
       'link sosmed', 'lowongan', 'lupa password', 'nama', 'pas foto',
       'pengantar', 'penutup', 'qualification', 'responsibilities',
       'salary', 'sapa', 'sertifikat', 'skck', 'timeline', 'training',
       'transkrip-ijazah'], dtype=object)

In [228]:
textvect('loker')

<tf.Tensor: shape=(8,), dtype=int64, numpy=array([151,   0,   0,   0,   0,   0,   0,   0], dtype=int64)>

In [230]:
textvect('wis')

<tf.Tensor: shape=(8,), dtype=int64, numpy=array([1, 0, 0, 0, 0, 0, 0, 0], dtype=int64)>